## Imports

In [1]:
import sys
import os
import pandas as pd
import time
import math
import random
import itertools

## Generating Solutions

In [2]:
class Generate_Routes():
    
    def __init__(self, number_of_nodes, max_distance_travel, nodes_time_window, time_between_nodes, distance_between_nodes, unloading_time):
        
        self.time_window = []
        self.pruned_routes = []
        self.unloading_time  = []
        self.travel_time = []
        self.travel_distance = []
        self.number_of_nodes = number_of_nodes
        self.max_distance_travel = max_distance_travel

        self.unloading_time_path = unloading_time
        self.node_time_window_path = nodes_time_window
        self.time_between_nodes_path = time_between_nodes
        self.distance_between_nodes_path = distance_between_nodes

        
        self.set_travel_time()
        self.set_time_windows()
        self.set_unloading_time()
        self.set_travel_distance()
        self.run()
        
    
    '''
    Some Important Variables
    '''
    
    
    # Time Window
    def set_time_windows(self):
        nodes_time_window  = self.node_time_window_path
        with open(nodes_time_window, 'r') as f:
            for line in f:
                window_start, window_end = map(int,line.rstrip().split())
                self.time_window.append((window_start, window_end))
            f.close()

    # Unloading Time
    def set_unloading_time(self):
        with open(self.unloading_time_path,'r') as f:
            self.unloading_time  = list(map(int, f.readline().split()))
    
    # Distance Traveled
    def set_travel_distance(self):
        distance_data        = pd.read_csv(self.distance_between_nodes_path)
        distance_data        = distance_data.iloc[:self.number_of_nodes, 1:self.number_of_nodes+1]
        travel_distance      = distance_data.values.tolist()
        self.travel_distance = travel_distance
    
    # Travel Time
    def set_travel_time(self):
        travel_time_data = pd.read_csv(self.time_between_nodes_path)
        travel_time_data = travel_time_data.iloc[:self.number_of_nodes, 1:self.number_of_nodes+1]
        travel_time      = travel_time_data.values.tolist()
        travel_time      = [list(map(int, x)) for x in travel_time] #Converting to integer
        self.travel_time = travel_time

    
    ''' 
    
    Algorithm Functions
    
    '''
    
    def generateAllTours(self):
        ''' Generate tours '''
        nodes = [i for i in range(1, self.number_of_nodes)]
        tours_with_2_stops = list(itertools.permutations(nodes, 2))

        return tours_with_2_stops
    
    
    def isFeasibleTimeWindow(self, hub, stop_1, stop_2):
        
        '''
        Time_window[hub] + travel_time[hub][stop1]
        Time_window[stop1] travel_time[stop1][stop2] + time_window[stop2]
        '''
        
        ### Starting from hub, traveling to stop-1, unloading before the end of time window at stop-1
        if(self.time_window[hub][0] + self.travel_time[hub][stop_1] + self.unloading_time[stop_1] > self.time_window[stop_1][1]):
            return False
        
        ### Starting after unloading at stop-1, traveling to stop-2, 
        ### unloading before the end of time window at stop-2
        elif(self.time_window[stop_1][0] + self.unloading_time[stop_1] + self.travel_time[stop_1][stop_2] 
             + self.unloading_time[stop_2] > self.time_window[stop_2][1]):
            return False
        
        #Add more conditions if needed
        return True
        
    def pruneBasedOnTimeWindows(self, tours):
        hub = 0
        for tour in tours:
            stop_1, stop_2 = tour

            if self.isFeasibleTimeWindow(hub, stop_1, stop_2):
                self.pruned_routes.append([hub, stop_1, stop_2])
        return
    
    def isFeasibleDistance(self, hub, stop_1, stop_2):
        ''' Distance travelled by truck during tour should be less than max allowed distance'''
        
        if(self.distance_travel[hub][stop_1] + self.distance_travel[stop_1][stop_2] > self.max_distance_travel):
            return False
        return True
    
    def pruneBasedOnDistance(self):
        tours = self.pruned_routes
        for tour in tours:
            hub, stop_1, stop_2 = tour
            
            if self.isFeasibleDistance:
                continue
            else:
                self.pruned_routes.remove([hub, stop_1, stop_2])
            return
            
            
    def run(self):
        two_stops = self.generateAllTours()
        self.pruneBasedOnTimeWindows(two_stops)
        self.pruneBasedOnDistance()
        print("All Routes Generated")

### Printing to file and formatting it

In [3]:
def print_routes_to_file(pruned_routes, pruned_routes_path):
    with open(pruned_routes_path, "w") as f:
        for route in pruned_routes:
            hub, stop_1, stop_2 = route
            f.write(str(hub) + " " + str(stop_1) + " " + str(stop_2) + "\n")
        f.close()

def format_obtained_routes(number_of_stations, final_routing_table_path, pruned_routes):
        
    #Table Header
    with open(final_routing_table_path, 'w') as f:
        f.write("Routes" + "," + "Stop_1" + "," + "Stop_2" + "\n")
        
        for i in range(number_of_stations):
            f.write(str(i) + "," + "S" + str(i) + "\n")
        f.close()
    
    with open(pruned_routes, 'r') as f:
        current_station = number_of_stations #just a counter
        
        for line in f:
            hub, stop_1, stop_2 = line.split()
            route_string = "S{},S{}\n".format(stop_1, stop_2)
            
            with open(final_routing_table_path, 'a') as f2:
                f2.write(str(current_station) + "," + route_string)
                current_station += 1
                f2.close()
        f.close()

## Main Function

Set paths and change variables here, then execute the next cell

In [4]:
def main():
    
    
    nodes_time_window      = "data/nodes_time_window.txt"
    time_between_nodes     = "data/site_time.csv"
    distance_between_nodes = "data/Distance_Matrix_File.csv"
    unloading_time_path    = "data/unloading_time.txt"
    
    pruned_routes_path       = "data/pruned_routes.txt"
    final_routing_table_path = "data/final_routing_table.csv"
    demand_constraints_path  = "data/demand_delivery_contraint.csv"

    

    number_of_nodes = 19
    max_distance_travel = 1000000
    
    #Generate solution
    sol = Generate_Routes(number_of_nodes, max_distance_travel, nodes_time_window, time_between_nodes, distance_between_nodes, unloading_time_path)
    
    #Printing to file and formatting
    print_routes_to_file(sol.pruned_routes, pruned_routes_path)

    #Format data for Inventory Rotuing
    format_obtained_routes(sol.number_of_nodes - 1, final_routing_table_path, pruned_routes_path)

    '''
    Files created:
        - data/pruned_routes.txt
        - data/final_routing_table.csv
    '''
    
    # Inventory Routing Related
    # routes = pd.read_csv(final_routing_table_path).to_numpy()
    # demand = pd.read_csv(demand_constraints_path).to_numpy()
    

### Calling the main function

In [5]:
main()

All Routes Generated


In [6]:
number_of_nodes      = 19
distance_data        = pd.read_csv("data/Distance_Matrix_File.csv")
distance_data        = distance_data.iloc[:number_of_nodes, :number_of_nodes]
travel_distance      = distance_data.values.tolist()

In [8]:
distance_of_route = []
for stop in range(number_of_nodes):
    distance_of_route.append(travel_distance[0][stop])

with open("data/pruned_routes.txt", 'r') as f:
    for line in f:
        hub, stop_1, stop_2 = list(map(int,line.split())) 
        route_distance = travel_distance[hub][stop_1] + travel_distance[stop_1][stop_2]
        distance_of_route.append(route_distance)
#         print(hub, stop_1, stop_2, route_distance)

In [19]:
def cost_of_route(distance_of_route, multi_stop_route_factor = 1.5 ,cost_per_mile = 0.01 ,fixed_cost = 1):
    
    '''Variables'''
    total_cost = []
    
    for route in range(len(distance_of_route)):
        total_cost.append(fixed_cost + (distance_of_route[route] * cost_per_mile))
        
        if(route > number_of_nodes):
            total_cost[route] *= multi_stop_route_factor
    return total_cost

route_cost = cost_of_route(distance_of_route)

# Inventory Routing Formulation 


<b>Decision Variable:</b>

\begin{equation}
\begin{array}{11}
\ Y_{i,k} & \forall i \in R, k \in WD
\end{array}
\end{equation}
\begin{equation}
\begin{array}{11}
\ X_{i,j,k} & \forall i \in R, j \in S, k \in WD
\end{array}
\end{equation}

<b>Objective Function:</b>

\begin{equation}
\begin{array}{ll}
\text{minimise} &  \sum_{i \in R}  \sum_{k \in W} Y_{i,k}  c_{i,k}\\
\end{array}
\end{equation}


<b>Subject to:</b>
\
<i>Respecting Demand Constraints</i>

\begin{equation}
\begin{array}{11}
\sum_{i \in R} \sum_{k \in W}  X_{i,j,k} m_{i,j} = d_j & \forall j \in S
\end{array}
\end{equation}

\
<i>Respecting Delivery Acceptance Constraints</i>

\begin{equation}
\begin{array}{11}
\sum_{i \in R} X_{i,j,k} m_{i,j} \le a_j & \forall j \in S, k \in W
\end{array}
\end{equation}

\
<i>Respecting Truck Constraints</i>

\begin{equation}
\begin{array}{11}
\sum_{j \in S} X_{i,j,k}m_{i,j} \le C_iY_{i,k} & \forall i \in R, k \in W
\end{array}
\end{equation}



\
<b>General Representations</b>

\
<i>Sets</i>
* <i>R</i>: Set of routes
* <i>S</i>: Set of stores
* <i>W</i>: Set of days in a week

\
<i>Parameters</i>
\
$
\begin{equation} 
\begin{array}{11}
d_j & \text{: Weekly demand of store j,} & j \in S  
\end{array}
\end{equation}
$

$
\begin{equation} 
\begin{array}{11}
a_j & \text{: Delivery acceptance limits for store j,} & j \in S  
\end{array}
\end{equation}
$

$
\begin{equation} 
\begin{array}{11}
m_{i,j} & \text{: Dummy variable to indicate 1 if store j is along route i and 0 otherwise,} & i \in R, j \in S  
\end{array}
\end{equation}
$

$
\begin{equation} 
\begin{array}{11}
c_{i,k} & \text{: Cost of  truck on each route i on day k } 
\end{array}
\end{equation}
$

$
\begin{equation} 
\begin{array}{11}
C_{i} & \text{: Full capacity of truck on route i on each day  } 
\end{array}
\end{equation}
$

\
<i>Variables</i>
$
\begin{equation} 
\begin{array}{11}
X_{i,j,k} & \text{: Units on each route i to be delivered to a store j on day k} \\
Y_{i,k} & \text{: If route i is used on day k}
\end{array}
\end{equation}
$

\
<i>Entities</i>
* <i>i</i>: Route in R
* <i>j</i>: Store in S
* <i>k</i>: Day in W


In [12]:
#Importing Packages

from gurobipy import *
from math import sqrt
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import xlrd

In [13]:
#Importing Data

routes = pd.read_csv("data/final_routing_table.csv").to_numpy()
print(routes)

demand = pd.read_csv("data/demand_delivery_contraint.csv").to_numpy()
print(demand)


[[0 'S0' nan]
 [1 'S1' nan]
 [2 'S2' nan]
 [3 'S3' nan]
 [4 'S4' nan]
 [5 'S5' nan]
 [6 'S6' nan]
 [7 'S7' nan]
 [8 'S8' nan]
 [9 'S9' nan]
 [10 'S10' nan]
 [11 'S11' nan]
 [12 'S12' nan]
 [13 'S13' nan]
 [14 'S14' nan]
 [15 'S15' nan]
 [16 'S16' nan]
 [17 'S17' nan]
 [18 'S1' 'S2']
 [19 'S1' 'S3']
 [20 'S1' 'S4']
 [21 'S1' 'S5']
 [22 'S1' 'S6']
 [23 'S1' 'S7']
 [24 'S1' 'S8']
 [25 'S1' 'S9']
 [26 'S1' 'S10']
 [27 'S1' 'S11']
 [28 'S1' 'S12']
 [29 'S1' 'S13']
 [30 'S1' 'S14']
 [31 'S1' 'S15']
 [32 'S1' 'S16']
 [33 'S1' 'S17']
 [34 'S2' 'S1']
 [35 'S2' 'S3']
 [36 'S2' 'S4']
 [37 'S2' 'S5']
 [38 'S2' 'S6']
 [39 'S2' 'S7']
 [40 'S2' 'S8']
 [41 'S2' 'S9']
 [42 'S2' 'S10']
 [43 'S2' 'S11']
 [44 'S2' 'S12']
 [45 'S2' 'S13']
 [46 'S2' 'S14']
 [47 'S2' 'S15']
 [48 'S2' 'S16']
 [49 'S2' 'S17']
 [50 'S3' 'S1']
 [51 'S3' 'S2']
 [52 'S3' 'S4']
 [53 'S3' 'S5']
 [54 'S3' 'S6']
 [55 'S3' 'S7']
 [56 'S3' 'S8']
 [57 'S3' 'S9']
 [58 'S3' 'S10']
 [59 'S3' 'S11']
 [60 'S3' 'S12']
 [61 'S3' 'S13']
 [62 'S3

In [16]:
#Demand
d = demand[:,1]

#Delivery constraints
dc = demand[:,2]

#Route matrix
#Store names 
snames = demand[:,0]

count = 1
num_routes = len(routes)
num_stores = len(demand)
mat = np.zeros(num_routes)
for i in snames:
    routestore1 = (routes[:,1] == str(i)) * 1
    routestore2 = (routes[:,2] == str(i)) * 1
    mainroutestore = routestore1 + routestore2
    mat = np.row_stack((mat, mainroutestore))
    #mat = np.concatenate((mat, mainroutestore),axis=1)
    
mat = np.array(mat, dtype='int16')[1:,:].transpose((1,0))

print(mat)


[[1 0 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 1 0 1]
 [0 0 0 ... 0 1 1]]


In [17]:
#Shipping costs
c = np.full((num_routes, num_stores), 1000)
c

#Supply constraints (truck capacity)
t = np.full((num_routes,7), 30)
t

array([[30, 30, 30, ..., 30, 30, 30],
       [30, 30, 30, ..., 30, 30, 30],
       [30, 30, 30, ..., 30, 30, 30],
       ...,
       [30, 30, 30, ..., 30, 30, 30],
       [30, 30, 30, ..., 30, 30, 30],
       [30, 30, 30, ..., 30, 30, 30]])

In [20]:
#Transportation Problem
m4 = Model('transportation')

#Variables
#Variables are in proportion
var = m4.addVars(num_routes,num_stores,7)
yvar = m4.addVars(num_routes,7,vtype = GRB.BINARY)

#Objective
m4.setObjective(sum(yvar[i,k] * route_cost[i] for i in range(num_routes) for k in range(7)), GRB.MINIMIZE)

#Weekly Demand
for j in range(num_stores):
    m4.addConstr(sum(var[i,j,k]*mat[i,j] for i in range(num_routes) for k in range(7)) == d[j])
    
#Delivery Constraints
for j in range(num_stores):
    for k in range(7):
        m4.addConstr(sum(var[i,j,k]*mat[i,j] for i in range(num_routes)) <= dc[j]*sum(yvar[i,k]*mat[i,j] for i in range(num_routes)))

#Supply constraint
for i in range(num_routes):
    for k in range(7):
        m4.addConstr(sum(var[i,j,k]*mat[i,j] for j in range(num_stores)) <= t[i,k]*yvar[i,k])
        
#Solving the optimization problem
m4.optimize()




Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 2329 rows, 43540 columns and 19089 nonzeros
Model fingerprint: 0xb39931df
Variable types: 41363 continuous, 2177 integer (2177 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [1e+00, 4e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+01, 5e+02]
Found heuristic solution: objective 86501.235000
Presolve removed 127 rows and 37149 columns
Presolve time: 0.07s
Presolved: 2202 rows, 6391 columns, 11382 nonzeros
Variable types: 4221 continuous, 2170 integer (2170 binary)

Root relaxation: objective 1.891175e+04, 3963 iterations, 0.04 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 18911.7521    0   18 86501.2350 18911.7521  78.1%     -    0s
H    0     0                

In [21]:
sum(yvar[i,2].getAttr("x")*mat for i in range(num_routes))


array([[19.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0., 19.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0., 19., ...,  0.,  0.,  0.],
       ...,
       [ 0.,  0.,  0., ...,  0.,  0., 19.],
       [ 0.,  0.,  0., ..., 19.,  0., 19.],
       [ 0.,  0.,  0., ...,  0., 19., 19.]])

In [22]:

#Printing the optimal solutions obtained
print("Optimal Solutions:")
for i, val in var.items():
    if val.getAttr("x") != 0:
        print("Number of units from route %g to store %g on day %g:\t %g " %(i[0]+1, i[1]+1, i[2]+1, val.getAttr("x")))
        
        
#Printing y
for i, val in yvar.items():
    print("Run route %g on day %g:\t %g " %(i[0]+1, i[1]+1, val.getAttr("x")))
    
print(yvar)
 
for i in range(num_routes):
    for k in range(7):
        print(yvar[i,k].getAttr("x"))


Optimal Solutions:
Number of units from route 3 to store 3 on day 1:	 7.0348 
Number of units from route 3 to store 3 on day 2:	 30 
Number of units from route 3 to store 3 on day 3:	 30 
Number of units from route 3 to store 3 on day 4:	 30 
Number of units from route 3 to store 3 on day 5:	 30 
Number of units from route 3 to store 3 on day 7:	 30 
Number of units from route 4 to store 4 on day 1:	 30 
Number of units from route 4 to store 4 on day 2:	 30 
Number of units from route 4 to store 4 on day 3:	 30 
Number of units from route 4 to store 4 on day 4:	 30 
Number of units from route 4 to store 4 on day 5:	 30 
Number of units from route 4 to store 4 on day 6:	 30 
Number of units from route 4 to store 4 on day 7:	 30 
Number of units from route 5 to store 5 on day 6:	 18.0586 
Number of units from route 6 to store 6 on day 1:	 30 
Number of units from route 6 to store 6 on day 2:	 30 
Number of units from route 6 to store 6 on day 3:	 30 
Number of units from route 6 to store

# Time Windows Optimisation

Set of optimal routes -> `j`

Number of days -> `k = 7`

Numer of time slots -> `t = 48`

In [23]:
number_of_nodes      = 20

all_routes = []
optimal_routes = []
unloading_time = []
optimal_routes_number = []

d_j1 = []
d_j2 = []

lower_bounds = []
upper_bounds = []

lower_bound_stop_1 = []
lower_bound_stop_2 = []
upper_bound_stop_1 = []
upper_bound_stop_2 = []

lower_bound_hub = []
upper_bound_hub = []

Y_jk = []


In [24]:
with open("data/unloading_time.txt", 'r') as f:
    unloading_time = list(map(int, f.readline().split()))
    
distance_data        = pd.read_csv("data/site_time.csv")
distance_data        = distance_data.iloc[:number_of_nodes, 1:number_of_nodes+1]
travel_distance      = distance_data.values.tolist()

- Mapping optimal route number --> actual route number
- Optimal_route_number[n] returns the index corresponding to the n^th optimal route

- `optimal_routes_number[j]` --> `i`

In [25]:
for i in range(num_routes):
    for k in range(7):
        if(yvar[i,k].getAttr("x") != 0):
            optimal_routes_number.append(i)
            break

In [26]:
with open("data/pruned_routes.txt", 'r') as f:
    for line in f:
        hub, stop_1, stop_2 = list(map(int,line.split()))
        all_routes.append([hub, stop_1, stop_2])

for index in optimal_routes_number:
    optimal_routes.append(all_routes[index])

In [27]:
for hub, stop_1, stop_2 in optimal_routes:
    d_j1.append(travel_distance[hub][stop_1])
    d_j2.append(travel_distance[hub][stop_1] + unloading_time[stop_1] + travel_distance[stop_1][stop_2])

In [28]:
for route_number in optimal_routes_number:
    day = []
    for weekday in range(7):
        if(yvar[route_number,weekday].getAttr("x") != 0):
            day.append(1)
        else:
            day.append(0)
    Y_jk.append(day)

In [29]:
Y_jk

[[1, 1, 1, 1, 1, 0, 1],
 [1, 1, 1, 1, 1, 1, 1],
 [0, 0, 0, 0, 0, 1, 0],
 [1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1],
 [1, 0, 1, 1, 0, 1, 0],
 [1, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 1, 1, 0],
 [0, 1, 0, 1, 0, 1, 1],
 [0, 0, 1, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 1],
 [1, 0, 1, 0, 1, 0, 0],
 [0, 0, 0, 0, 1, 0, 0],
 [1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1],
 [0, 0, 0, 0, 1, 0, 0],
 [1, 0, 0, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1],
 [0, 0, 0, 0, 1, 0, 0],
 [1, 1, 1, 1, 1, 0, 1],
 [1, 1, 0, 0, 1, 0, 0],
 [0, 1, 0, 0, 0, 0, 0],
 [1, 0, 1, 0, 1, 1, 0],
 [1, 1, 0, 0, 1, 0, 0],
 [0, 0, 1, 0, 0, 0, 0],
 [0, 0, 0, 0, 1, 0, 0]]

In [30]:
optimal_routes

[[0, 1, 4],
 [0, 1, 5],
 [0, 1, 6],
 [0, 1, 7],
 [0, 1, 8],
 [0, 1, 9],
 [0, 1, 10],
 [0, 1, 12],
 [0, 1, 13],
 [0, 1, 14],
 [0, 1, 15],
 [0, 1, 16],
 [0, 1, 17],
 [0, 2, 1],
 [0, 2, 3],
 [0, 4, 7],
 [0, 6, 7],
 [0, 6, 11],
 [0, 6, 12],
 [0, 7, 3],
 [0, 7, 11],
 [0, 8, 1],
 [0, 9, 7],
 [0, 9, 10],
 [0, 10, 1],
 [0, 10, 3],
 [0, 12, 15],
 [0, 13, 15],
 [0, 16, 8],
 [0, 16, 14],
 [0, 17, 1],
 [0, 18, 7]]

In [31]:
print(d_j1)

[1.407175, 1.407175, 1.407175, 1.407175, 1.407175, 1.407175, 1.407175, 1.407175, 1.407175, 1.407175, 1.407175, 1.407175, 1.407175, 1.75544375, 1.75544375, 0.3104875, 0.2923, 0.2923, 0.2923, 0.44930625, 0.44930625, 0.25653125, 0.9347875, 0.9347875, 1.80436875, 1.80436875, 1.6456375, 1.7986375, 0.36789375, 0.36789375, 0.2014375, 7.7930875]


In [32]:
lower_bounds = []
upper_bounds = []

lower_bound_stop_1 = []
lower_bound_stop_2 = []
upper_bound_stop_1 = []
upper_bound_stop_2 = []

lower_bound_hub = []
upper_bound_hub = []

with open("data/nodes_time_window.txt", 'r') as f:
    for line in f:
        lower_bound, upper_bound = map(int, line.split())
        lower_bounds.append(lower_bound)
        upper_bounds.append(upper_bound)

for hub, stop_1, stop_2 in optimal_routes:
    lower_bound_stop_1.append(lower_bounds[stop_1])
    lower_bound_stop_2.append(lower_bounds[stop_2])
    upper_bound_stop_1.append(upper_bounds[stop_1])
    upper_bound_stop_2.append(upper_bounds[stop_2])
    
    lower_bound_hub.append(lower_bounds[hub])
    upper_bound_hub.append(upper_bounds[hub])

In [33]:
print(lower_bound_stop_1)
print(lower_bound_stop_2)
print("\n")
print(upper_bound_stop_1)
print(upper_bound_stop_2)

[35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 17, 17, 17, 17]
[35, 35, 35, 35, 35, 35, 35, 35, 35, 17, 17, 17, 17, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 17, 17, 35, 17, 35, 35]


[48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 44, 44, 44, 44]
[48, 48, 48, 48, 48, 48, 48, 48, 48, 44, 44, 44, 44, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 44, 44, 48, 44, 48, 48]


## Optimisation Formulation
###### Please run all previous code starting here

In [34]:
num_routes = len(optimal_routes)
num_days   = 7
time_slots = 48

In [35]:
Y_jk = np.array(Y_jk)

#Time Windows Problem
m0 = Model('Time_Windows')

#Variables
#Variables are in proportion
Z_jkt = m0.addVars(num_routes, num_days, time_slots, vtype = GRB.BINARY)
slack = m0.addVars(num_days, time_slots, vtype = GRB.CONTINUOUS)

#Objective - Minimise Total Slack
m0.setObjective(sum(slack[k,t] for k in range(num_days) for t in range(time_slots)), GRB.MINIMIZE)

#Use a route only once in a day
for j in range(num_routes):
    for k in range(num_days):
        m0.addConstr(sum(Z_jkt[j,k,t] for t in range(time_slots)) == 1)

#Maximum slack allowed contraint
for k in range(num_days):
    for t in range(time_slots):
        m0.addConstr(sum(Z_jkt[j,k,t] for j in range(num_routes)) <= 3 + slack[k,t])

# #Route is chosen zjkt <= yjk
for j in range(num_routes):
    for k in range(num_days):
        for t in range(time_slots):
            m0.addConstr(Z_jkt[j,k,t] <= Y_jk[j, k])

# #Upper Bounds and Lower Bounds
# #-# 1. Hub 
for j in range(num_routes):
    for k in range(num_days):
        m0.addConstr(sum(t * Z_jkt[j,k,t] for t in range(time_slots)) 
                     <= upper_bound_hub[j])
        
for j in range(num_routes):
    for k in range(num_days):
        m0.addConstr(lower_bound_hub[j] <= sum(t * Z_jkt[j,k,t] for t in range(time_slots)) 
                     )
    
#-# 2. Stop_1
for j in range(num_routes):
    for k in range(num_days):
        m0.addConstr(sum(t * Z_jkt[j,k,t] for t in range(time_slots)) + d_j1[j] 
                     <= upper_bound_stop_1[j])
        
for j in range(num_routes):
    for k in range(num_days):
        m0.addConstr(lower_bound_stop_1[j] <= sum(t * Z_jkt[j,k,t] for t in range(time_slots)) + d_j1[j] 
                     )
#-# 3. Stop_2
for j in range(num_routes):
    for k in range(num_days):
        m0.addConstr(lower_bound_stop_2[j] <= sum(t * Z_jkt[j,k,t] for t in range(time_slots)) + d_j2[j] 
                     )

for j in range(num_routes):
    for k in range(num_days):
        m0.addConstr(sum(t * Z_jkt[j,k,t] for t in range(time_slots)) + d_j2[j] 
                     <= upper_bound_stop_2[j])
        
        
#Solving the optimization problem
m0.optimize()



#Printing the optimal solutions obtained
# print("Optimal Solutions:")
# for i, val in var.items():
#     if val.getAttr("x") != 0:
#         print("Number of units from route %g to store %g on day %g:\t %g " %(i[0]+1, i[1]+1, i[2]+1, val.getAttr("x")))
        
        
# #Printing y
# for i, val in yvar.items():
#     print("Run route %g on day %g:\t %g " %(i[0]+1, i[1]+1, val.getAttr("x")))
    
# print(yvar)

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 12656 rows, 11088 columns and 95760 nonzeros
Model fingerprint: 0x024c3f09
Variable types: 336 continuous, 10752 integer (10752 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+01]
Presolve removed 6480 rows and 0 columns
Presolve time: 0.01s

Explored 0 nodes (0 simplex iterations) in 0.11 seconds
Thread count was 1 (of 4 available processors)

Solution count 0

Model is infeasible
Best objective -, best bound -, gap -
